In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style = 'whitegrid')

import os

data_path = 'C:\\Users\\sunil\\Projects\\Analytics vidya\\Finhack\\Data\\'
submission_path = 'C:\\Users\\sunil\\Projects\\Analytics vidya\\Finhack\\Data\\submission\\'
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

In [2]:
pd.set_option('display.max_columns', 100)

In [3]:
%%time
train_data = pd.read_excel(data_path + 'Train\\Train\\train_Data.xlsx')
train_bureau = pd.read_excel(data_path + 'Train\\Train\\train_bureau.xlsx')

test_data = pd.read_excel(data_path + 'Test\\test_Data.xlsx')
test_bureau = pd.read_excel(data_path + 'Test\\test_bureau.xlsx')

Wall time: 2min 30s


In [4]:
sample_sub = pd.read_csv(data_path + 'sample_submission_ejm25Dc.csv')

In [6]:
target = 'Top-up Month'
ID = 'ID'
d_date = 'DisbursalDate'
m_date = 'MaturityDAte'
a_date = 'AuthDate'
features = [col for col in train_data.columns if col not in [target, ID, d_date, m_date, a_date]]

In [7]:
train_data[target].unique()

array([' > 48 Months', 'No Top-up Service', '12-18 Months',
       '36-48 Months', '18-24 Months', '24-30 Months', '30-36 Months'],
      dtype=object)

In [8]:
train_target = [' > 48 Months', 'No Top-up Service', '12-18 Months','36-48 Months',
                '18-24 Months', '24-30 Months', '30-36 Months']
map_ls = [0, 1, 2, 3, 4, 5, 6]
map_dict = dict(zip(train_target, map_ls))
reverse_map_dict = dict(zip(map_ls, train_target))
train_data[target] = train_data[target].map(map_dict)

In [157]:
df = pd.concat( [train_data, test_data], axis = 0).reset_index(drop = True)

In [160]:
df['Area'].fillna(value = df['Area'].mode()[0], inplace = True)
df['ManufacturerID'].fillna(value = df['ManufacturerID'].mode()[0], inplace = True)
df['SEX'].fillna(value = df['SEX'].mode()[0], inplace = True)
df['AGE'].fillna(value = df['AGE'].mode()[0], inplace = True)
df['MonthlyIncome'].fillna(value = df['MonthlyIncome'].mode()[0], inplace = True)
df['City'].fillna(value = df['City'].mode()[0], inplace = True)
df['City'].fillna(value = df['City'].mode()[0], inplace = True)
df['ZiPCODE'].fillna(value = df['ZiPCODE'].mode()[0], inplace = True)

In [161]:
cat_cols = [ 'Frequency', 'InstlmentMode', 'LoanStatus', 'PaymentMode','Area', 
            'DisbursalDate','AssetID', 'ManufacturerID', 'SupplierID', 'LTV', 
            'SEX', 'City', 'State', 'ZiPCODE']

In [162]:
df[cat_cols] = df[cat_cols].apply(le.fit_transform)

In [163]:
train_proc, test_proc = df.iloc[:train_data.shape[0]], df.iloc[train_data.shape[0]:]

In [164]:
trn, val = train_test_split(train_proc, test_size=0.2, random_state = 1, stratify = train_proc[target])

###### Input to our model will be the features
X_trn, X_val = trn[features], val[features]

###### Output of our model will be the TARGET_COL
y_trn, y_val = trn[target], val[target]

##### Features for the test data that we will be predicting
X_test = test_proc[features]

In [191]:
lr = LogisticRegression(random_state = 1, solver = 'saga', max_iter = 100)
_ = lr.fit(X_trn, y_trn)

preds_val = lr.predict(X_val)

f1_score(y_val, preds_val, average='macro')

C:\Users\sunil\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


0.1295141776593355

In [180]:
clf = LGBMClassifier(random_state = 1,)
_ = clf.fit(X_trn, y_trn)

preds_val = clf.predict(X_val)

f1_score(y_val, preds_val, average='macro')

0.19121508839603177

In [ ]:
preds = clf.predic(X_test)

In [182]:
sample_sub[target] = preds

In [183]:
sample_sub[target] = sample_sub[target].map(reverse_map_dict)

In [184]:
sample_sub.to_csv(submission_path + 'LGBM_baseline_without_merge.csv', index = False)

In [185]:
sample_sub

,ID,Top-up Month
0,4,No Top-up Service
1,5,No Top-up Service
2,6,No Top-up Service
3,25,No Top-up Service
4,119,No Top-up Service
...,...,...
14740,143396,No Top-up Service
14741,143397,No Top-up Service
14742,143398,No Top-up Service
14743,143399,No Top-up Service


---
# Merged

In [9]:
merged_train = pd.merge(left=train_data, right=train_bureau, left_on='ID', right_on='ID')
merged_test = pd.merge(left=test_data, right=test_bureau, left_on='ID', right_on='ID')

In [10]:
df = pd.concat( [merged_train, merged_test], axis = 0).reset_index(drop = 0)

In [11]:
cat_cols = [ 'Frequency', 'InstlmentMode', 'LoanStatus', 'PaymentMode','Area', 
            'DisbursalDate','AssetID', 'ManufacturerID', 'SupplierID', 'LTV', 
            'SEX', 'City', 'State', 'ZiPCODE', 'Frequency', 'InstlmentMode', 
            'LoanStatus', 'PaymentMode','BranchID', 'Area', 'Tenure','AssetID', 
            'ManufacturerID', 'SupplierID', 'LTV', 'SEX','AGE', 'City', 'State',
            'ZiPCODE', 'Top-up Month','SELF-INDICATOR', 'MATCH-TYPE','ACCT-TYPE', 'OWNERSHIP-IND', 'ACCOUNT-STATUS']

In [20]:
df['Area'].fillna(value = df['Area'].mode()[0], inplace = True)
df['ManufacturerID'].fillna(value = df['ManufacturerID'].mode()[0], inplace = True)
df['SEX'].fillna(value = df['SEX'].mode()[0], inplace = True)
df['AGE'].fillna(value = df['AGE'].mode()[0], inplace = True)
df['MonthlyIncome'].fillna(value = df['MonthlyIncome'].mode()[0], inplace = True)
df['City'].fillna(value = df['City'].mode()[0], inplace = True)
df['ZiPCODE'].fillna(value = df['ZiPCODE'].mode()[0], inplace = True)
df['MaturityDAte'].fillna(value = df['MaturityDAte'].mode()[0], inplace = True)
df['DISBURSED-AMT/HIGH CREDIT'].fillna(value = df['DISBURSED-AMT/HIGH CREDIT'].mode()[0], inplace = True)
df['CURRENT-BAL'].fillna(value = df['CURRENT-BAL'].mode()[0], inplace = True)
df['WRITE-OFF-AMT'].fillna(value = df['WRITE-OFF-AMT'].mode()[0], inplace = True)
df['CURRENT-BAL'].fillna(value = df['CURRENT-BAL'].mode()[0], inplace = True)
df['OVERDUE-AMT'].fillna(value = df['OVERDUE-AMT'].mode()[0], inplace = True)

In [ ]:
'ASSET_CLASS','REPORTED DATE - HIST', 'DPD - HIST', 'CUR BAL - HIST', 'AMT OVERDUE - HIST', 'AMT PAID - HIST', 'TENURE'

In [34]:
target = 'Top-up Month'
ID = 'ID'
d_date = 'DisbursalDate'
m_date = 'MaturityDAte'
a_date = 'AuthDate'
m_d_date = 'DATE-REPORTED'
m_dd_date = 'DISBURSED-DT'
m_c_date = 'CLOSE-DT'
m_l_date = 'LAST-PAYMENT-DATE'
m_ddd_date = 'DISBURSED-AMT/HIGH CREDIT'

features = [col for col in df.columns if col not in [target, ID, d_date, m_date, a_date, m_d_date, m_dd_date, m_ddd_date, m_c_date, m_l_date, 'ASSET_CLASS','REPORTED DATE - HIST', 'DPD - HIST', 'CUR BAL - HIST', 'AMT OVERDUE - HIST', 'AMT PAID - HIST', 'TENURE', 'CREDIT-LIMIT/SANC AMT', 'INSTALLMENT-AMT', 'INSTALLMENT-FREQUENCY', 'CONTRIBUTOR-TYPE', 'CURRENT-BAL', 'OVERDUE-AMT']]

In [35]:
'CONTRIBUTOR-TYPE', 'CURRENT-BAL', 'OVERDUE-AMT'

('CONTRIBUTOR-TYPE', 'CURRENT-BAL', 'OVERDUE-AMT')

In [36]:
df[cat_cols] = df[cat_cols].apply(le.fit_transform)

In [37]:
train_proc, test_proc = df.iloc[:merged_train.shape[0]], df.iloc[merged_train.shape[0]:]

In [38]:
trn, val = train_test_split(train_proc, test_size=0.2, random_state = 1, stratify = train_proc[target])

###### Input to our model will be the features
X_trn, X_val = trn[features], val[features]

###### Output of our model will be the TARGET_COL
y_trn, y_val = trn[target], val[target]

##### Features for the test data that we will be predicting
X_test = test_proc[features]

In [39]:
clf = LGBMClassifier(random_state = 1,)
_ = clf.fit(X_trn, y_trn)

preds_val = clf.predict(X_val)

f1_score(y_val, preds_val, average='macro')

0.5545859918373336

In [41]:
preds = clf.predict(X_test)

In [44]:
merged_test['preds'] = preds

In [ ]:
source.groupby(['Country','City'])['Short name'].agg(pd.Series.mode).to_frame()

In [58]:
id_pred_dict = round(merged_test.groupby('ID')['preds'].median()).to_dict()

In [237]:
test_data.shape

(14745, 25)

In [59]:
sample_sub[target] = sample_sub['ID'].apply(lambda x: id_pred_dict[x])

In [60]:
sample_sub[target] = sample_sub[target].map(reverse_map_dict)

In [63]:
sample_sub.to_csv(submission_path + 'LGBM_baseline_without_merge.csv', index = False)

In [61]:
sample_sub[target].value_counts()

No Top-up Service    13686
12-18 Months           503
 > 48 Months           239
30-36 Months           140
24-30 Months            81
36-48 Months            63
18-24 Months            33
Name: Top-up Month, dtype: int64